In [2]:
import pandas as pd
from datetime import datetime

# Read the CSV files
# Note: Adjust the file paths as needed
df_project = pd.read_csv('/Users/marc/Desktop/DS project M2 & Asset Prices - Group D(Variables).csv', sep=';')
df_gold = pd.read_csv('/Users/marc/Desktop/Gold Futures Historical Datacsv.csv')

# Clean up the project dataframe - remove empty first column and clean column names
df_project = df_project.iloc[:, 1:]  # Remove first empty column
df_project.columns = df_project.columns.str.strip()

# Convert date columns to datetime format
# Project file uses DD.MM.YYYY format
df_project['Date'] = pd.to_datetime(df_project['Date'], format='%d.%m.%Y')

# Gold file uses DD.MM.YYYY format
df_gold['Date'] = pd.to_datetime(df_gold['Date'], format='%d.%m.%Y')

print(f"Original DS Project Data rows: {len(df_project)}")
print(f"Original Gold Futures Data rows: {len(df_gold)}")

# Get the dates from DS Project file (this is our reference)
project_dates = set(df_project['Date'])

# Filter Gold Futures to keep only dates that exist in DS Project file
df_gold_matched = df_gold[df_gold['Date'].isin(project_dates)].copy()

# Sort both dataframes by date (most recent first)
df_project = df_project.sort_values('Date', ascending=False).reset_index(drop=True)
df_gold_matched = df_gold_matched.sort_values('Date', ascending=False).reset_index(drop=True)

print(f"\nMatched dates found: {len(df_gold_matched)}")
print(f"DS Project Data rows (unchanged): {len(df_project)}")
print(f"Gold Futures Data rows (after matching): {len(df_gold_matched)}")

# Check if all project dates have matching gold futures data
missing_in_gold = len(df_project) - len(df_gold_matched)
if missing_in_gold > 0:
    print(f"\nWarning: {missing_in_gold} dates from DS Project were NOT found in Gold Futures data")
    missing_dates = project_dates - set(df_gold_matched['Date'])
    print(f"\n=== Missing Dates (first 20) ===")
    for date in sorted(missing_dates)[:20]:
        print(date.strftime('%d.%m.%Y'))
    
    # Create a complete Gold Futures dataframe with all project dates
    # For missing dates, we'll add rows with NaN/empty values
    print(f"\n=== Adding Missing Dates to Gold Futures Data ===")
    
    # Create a dataframe with all project dates
    all_dates_df = pd.DataFrame({'Date': sorted(project_dates)})
    
    # Merge with gold futures data (left join to keep all dates)
    df_gold_complete = all_dates_df.merge(df_gold_matched, on='Date', how='left')
    
    # Sort by date (most recent first)
    df_gold_complete = df_gold_complete.sort_values('Date', ascending=False).reset_index(drop=True)
    
    print(f"Gold Futures Data with added missing dates: {len(df_gold_complete)} rows")
    
    # Save the complete version
    df_gold_complete.to_csv('Gold_Futures_complete_with_missing.csv', index=False)
    print("\nAdditional file saved:")
    print("- Gold_Futures_complete_with_missing.csv (includes all DS project dates, missing data shown as NaN)")
    
    # Show some examples of missing date rows
    missing_rows = df_gold_complete[df_gold_complete['Price'].isna()]
    if len(missing_rows) > 0:
        print(f"\n=== Sample of Missing Data Rows (first 5) ===")
        print(missing_rows[['Date']].head())
else:
    print("\n✓ All DS Project dates have matching Gold Futures data!")

# Save the matched dataframes
df_project.to_csv('DS_project_complete.csv', index=False)
df_gold_matched.to_csv('Gold_Futures_matched.csv', index=False)

print("\n=== Files Saved ===")
print("- DS_project_complete.csv (all original dates)")
print("- Gold_Futures_matched.csv (only dates matching DS project)")

# Display first few rows
print("\n=== First 5 rows of DS Project Data ===")
print(df_project[['Date']].head())

print("\n=== First 5 rows of Matched Gold Futures Data ===")
print(df_gold_matched[['Date', 'Price']].head())

# Show date range
print("\n=== Date Range ===")
print(f"DS Project - Earliest: {df_project['Date'].min()}, Latest: {df_project['Date'].max()}")
print(f"Gold Matched - Earliest: {df_gold_matched['Date'].min()}, Latest: {df_gold_matched['Date'].max()}")

Original DS Project Data rows: 1345
Original Gold Futures Data rows: 1350

Matched dates found: 1336
DS Project Data rows (unchanged): 1345
Gold Futures Data rows (after matching): 1336


=== Missing Dates (first 20) ===
05.07.2009
25.12.2011
01.01.2012
06.07.2014
05.07.2015
25.12.2016
01.01.2017
24.12.2017
31.12.2017

=== Adding Missing Dates to Gold Futures Data ===
Gold Futures Data with added missing dates: 1345 rows

Additional file saved:
- Gold_Futures_complete_with_missing.csv (includes all DS project dates, missing data shown as NaN)

=== Sample of Missing Data Rows (first 5) ===
          Date
405 2017-12-31
406 2017-12-24
457 2017-01-01
458 2016-12-25
535 2015-07-05

=== Files Saved ===
- DS_project_complete.csv (all original dates)
- Gold_Futures_matched.csv (only dates matching DS project)

=== First 5 rows of DS Project Data ===
        Date
0 2025-10-05
1 2025-09-28
2 2025-09-21
3 2025-09-14
4 2025-09-07

=== First 5 rows of Matched Gold Futures Data ===
        Date    